# VideoRobot — One-Click Colab Runner
این نوت‌بوک ریپو را کلون می‌کند، FFmpeg و وابستگی‌ها را نصب می‌کند، بک‌اند را **به‌صورت پکیج** بالا می‌آورد، یک مانیفست نمونه می‌سازد، رندر می‌زند و MP4 نهایی را برمی‌گرداند.

اگر قبلاً با اجرای مستقیم `backend/main.py` به خطای import نسبی خورده‌ای، این نسخه آن را با `python -m backend.main` حل می‌کند.

In [ ]:
# 1) Clone repo freshimport shutil, pathlibREPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"WORK = pathlib.Path("/content/videorobot")shutil.rmtree(WORK, ignore_errors=True)!git clone {REPO_URL} {WORK}!cd {WORK} && git rev-parse --short HEAD

In [ ]:
# 2) Install FFmpeg and Python deps (idempotent)!bash /content/videorobot/scripts/install_ffmpeg_colab.sh || (apt-get update && apt-get install -y ffmpeg)!pip install -r /content/videorobot/backend/requirements.txt

In [ ]:
# 3) Start backend as a PACKAGE and health-check (handles relative imports)import os, sys, time, requests, subprocess, pathlib, socketREPO = "/content/videorobot"BACKEND_DIR = pathlib.Path(REPO) / "backend"(BACKEND_DIR / "__init__.py").write_text("")  # ensure packagedef port_free(p):    s = socket.socket()    try:        s.bind(("127.0.0.1", p)); s.close(); return True    except OSError:        return Falsecand = [int(os.environ.get("BACKEND_PORT", "8000")), 8001, 8002]port = next((p for p in cand if port_free(p)), 8000)os.environ["BACKEND_PORT"] = str(port)print("Using BACKEND_PORT =", port)env = os.environ.copy(); env["PYTHONUNBUFFERED"] = "1"proc = subprocess.Popen([sys.executable, "-m", "backend.main"], cwd=REPO,                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)print("Backend PID:", proc.pid)base = f"http://127.0.0.1:{port}"# Stream logs while probing /healthz and /health, up to 180st0 = time.time(); healthy = False; logs = []while time.time() - t0 < 180:    try:        line = proc.stdout.readline()        if line:            logs.append(line);            logs = logs[-800:]            print(line, end="")    except Exception:        pass    for path in ("/healthz", "/health"):        try:            r = requests.get(base + path, timeout=2)            if r.ok:                print("Health OK via", path, ":", r.text); healthy = True; break        except Exception:            pass    if healthy: break    time.sleep(1)if not healthy:    print("\n❌ Not healthy; recent log tail:\n", "".join(logs[-200:]))    raise SystemExit("Backend still not healthy after 180s")print("\n✅ Backend healthy at", base)

In [ ]:
# 4) Create a simple manifestmanifest = {  "seed": 7,  "video": {"width": 720, "height": 1280, "fps": 30, "bg_color": "#101318"},  "tracks": [    {"type": "text", "content": "سلام دنیا", "start": 0.2, "duration": 3, "x": 40, "y": 80, "size": 72, "color": "#FFFFFF"}  ]}import json, pathlibpath = pathlib.Path("/content/manifest.json")path.write_text(json.dumps(manifest, ensure_ascii=False))print("Manifest:", path)

In [ ]:
# 5) Submit render, poll progress, and download resultimport requests, time, os, json, urllib.requestbase = f"http://127.0.0.1:{os.environ['BACKEND_PORT']}"resp = requests.post(base + "/render", json=json.loads(open("/content/manifest.json", "r", encoding="utf-8").read()), timeout=60)print("POST /render:", resp.status_code, resp.text)resp.raise_for_status()job_id = resp.json().get("job_id")assert job_id, "No job_id returned"for i in range(600):    r = requests.get(base + f"/progress/{job_id}", timeout=10)    if r.ok:        js = r.json(); print(f"tick {i}:", js.get("state"), js.get("pct"))        if js.get("state") == "success":            break        if js.get("state") == "error":            raise SystemExit(js)    time.sleep(1)out = f"/content/{job_id}.mp4"urllib.request.urlretrieve(base + f"/download?jobId={job_id}", out)print("Downloaded:", out)print("If not found here, also check /content/outputs/<job_id>/final.mp4")